<a href="https://colab.research.google.com/github/mkbahk/QuantumComputing/blob/main/QuantumAlgorithms_Application_HHL_Linear_Solver_mkbahk_20250911.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "qiskit[visualization]"
!pip install "qiskit-aer"
#!pip install "qiskit-terra"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 77.9 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=30d511d08977f6d493fb632ba4a322dc9dd66e03eca44645288329e6fd6d152c
  Stored in directory: /root/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 63.2 MB/s eta 0:00:00


In [7]:
%pip show qiskit qiskit-aer

Name: qiskit
Version: 2.1.2
Summary: An open-source SDK for working with quantum computers at the level of extended quantum circuits, operators, and primitives.
Home-page: https://www.ibm.com/quantum/qiskit
Author: 
Author-email: Qiskit Development Team <qiskit@us.ibm.com>
License: Apache 2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: dill, numpy, rustworkx, scipy, stevedore, typing-extensions
Required-by: qiskit-aer
---
Name: qiskit-aer
Version: 0.17.1
Summary: Aer - High performance simulators for Qiskit
Home-page: https://github.com/Qiskit/qiskit-aer
Author: AER Development Team
Author-email: qiskit@us.ibm.com
License: Apache 2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: numpy, psutil, python-dateutil, qiskit, scipy
Required-by: 


In [16]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.quantum_info import Statevector, Operator
from qiskit.circuit.library import UnitaryGate, QFT, RYGate

In [18]:
# 1. HHL 알고리즘의 구성 요소: 위상 추정 및 역 푸리에 변환
def apply_qpe(qc, num_eigen_qubits, A_op, t):
    """위상 추정(Phase Estimation)을 위한 양자 회로를 적용합니다."""
    qc.h(range(num_eigen_qubits))

    # A_op의 거듭제곱을 제어(controlled)하여 적용합니다.
    for i in range(num_eigen_qubits):
        power_of_2 = 2**i
        controlled_op = A_op.power(power_of_2)
        # 2x2 행렬의 경우, controlled_op은 3개의 큐비트에 적용됩니다.
        # (control_qubit, target_qubit, auxiliary_qubit)
        if controlled_op.num_qubits == 2:
            qc.append(controlled_op.control(), [i, num_eigen_qubits])
        else:
            # 더 큰 행렬의 경우, controlled_op은 여러 큐비트에 적용될 수 있습니다.
            # 이 샘플은 2x2 행렬에 맞춰져 있습니다.
            pass
        ###ㅑㄹ
    ###for

    qc.append(QFT(num_eigen_qubits, do_swaps=False).inverse(), range(num_eigen_qubits))
  ###def

In [19]:
def hhl_circuit(A, b):
    """HHL 알고리즘을 위한 양자 회로를 구성합니다."""

    # A 행렬의 차원 (2x2)
    n = int(np.log2(A.shape[0]))

    # 위상 추정을 위한 큐비트 수. 정밀도를 결정합니다.
    num_eigen_qubits = 2

    # 보조 큐비트 (1개)
    num_ancilla_qubits = 1

    # 총 큐비트 수
    num_qubits = num_ancilla_qubits + num_eigen_qubits + n
    # Adjust classical bits to measure ancilla and state qubits
    qc = QuantumCircuit(num_qubits, num_ancilla_qubits + n)


    # 큐비트 할당
    ancilla_qubit = 0
    eigen_qubits = range(1, 1 + num_eigen_qubits)
    state_qubits = range(1 + num_eigen_qubits, num_qubits)

    # 1. b 벡터 인코딩
    # b = [b0, b1] 벡터를 양자 상태로 인코딩합니다.
    # Statevector.from_label('00') 상태에 b 벡터를 적용합니다.
    qc.initialize(b / np.linalg.norm(b), state_qubits)

    # 2. 위상 추정 (Phase Estimation)
    # A 행렬을 양자 오퍼레이터로 변환합니다.
    A_op = UnitaryGate(A, label='A')

    # 이 부분은 샘플 코드의 복잡성을 줄이기 위해 단순화되었습니다.
    # 실제 HHL에서는 A를 e^(-iAt)와 같은 지수 형태로 변환해야 합니다.
    # Qiskit에서는 이 부분이 복잡하므로, 2x2 행렬에 대한 단순화된 예시를 사용합니다.

    # 3. 고윳값에 따른 조건부 회전
    # 위상 추정 큐비트의 상태에 따라 보조 큐비트를 회전시킵니다.
    # 이 부분은 1/lambda에 비례하는 회전입니다.
    # 여기서는 고윳값이 2와 1인 경우를 가정합니다.
    qc.ry(np.pi / 2, ancilla_qubit) # 임시 회전

    # 4. 역 위상 추정 (Inverse Phase Estimation)
    qc.append(QFT(num_eigen_qubits, do_swaps=False).inverse(), eigen_qubits)

    # 5. 첫 번째 큐비트(보조 큐비트) 측정
    qc.measure(ancilla_qubit, 0)

    # 6. 결과 상태 측정 (state_qubits into classical bits starting from index 1)
    qc.measure(state_qubits, range(1, 1 + n))


    return qc
###def

In [42]:
# 2. 문제 정의
# 해결하려는 선형방정식 Ax = b
A = np.array([[1, 0],
              [0, 1]])
b = np.array([1.5, 1.5])

# A 행렬의 고윳값은 4와 2입니다.
# 고유벡터는 각각 [1, 1]과 [1, -1]입니다.

# 3. HHL 회로 생성 및 실행
hhl_circ = hhl_circuit(A, b)
simulator = Aer.get_backend('aer_simulator')
tcirc = transpile(hhl_circ, simulator)
result = simulator.run(tcirc, shots=8192).result()
counts = result.get_counts(tcirc)

# 4. 결과 출력
print("====================================")
print("HHL 알고리즘 샘플")
print("------------------------------------")
print(f"문제: A * x = b")
print(f"A = \n{A}")
print(f"b = {b}")
print("------------------------------------")
print(f"시뮬레이터 측정 결과: {counts}")



HHL 알고리즘 샘플
------------------------------------
문제: A * x = b
A = 
[[1 0]
 [0 1]]
b = [1.5 1.5]
------------------------------------
시뮬레이터 측정 결과: {'01': 2085, '10': 2002, '00': 2073, '11': 2032}


In [43]:
# 5. 측정 결과 해석 및 해 추출
print("\n------------------------------------")
print("HHL 시뮬레이터 결과 해석")
print("------------------------------------")

# 보조 큐비트가 '1'로 측정된 결과만 필터링합니다.
# 측정 결과 키의 형식은 'ancilla_qubit state_qubits' 입니다.
# 여기서는 ancilla_qubit이 첫 번째 비트입니다.
hhl_results = {k: v for k, v in counts.items() if k[0] == '1'}

print(f"보조 큐비트가 '1'인 측정 결과: {hhl_results}")

# 필터링된 결과에서 상태 큐비트의 측정 횟수를 추출합니다.
# 상태 큐비트는 측정 결과 문자열의 나머지 비트입니다.
state_qubit_counts = {}
total_hhl_counts = sum(hhl_results.values())

if total_hhl_counts > 0:
    for measurement, count in hhl_results.items():
        state_measurement = measurement[1:] # 상태 큐비트 측정값
        if state_measurement in state_qubit_counts:
            state_qubit_counts[state_measurement] += count
        else:
            state_qubit_counts[state_measurement] = count

    print(f"상태 큐비트 측정 횟수 (보조 큐비트=1): {state_qubit_counts}")

    # 상태 큐bit 측정 횟수를 확률로 변환합니다.
    state_qubit_probabilities = {k: v / total_hhl_counts for k, v in state_qubit_counts.items()}
    print(f"상태 큐비트 확률 (보조 큐비트=1): {state_qubit_probabilities}")

    # 확률로부터 해 벡터의 상대적인 크기를 추정합니다.
    # 해 벡터의 요소는 해당 상태의 확률의 제곱근에 비례합니다.
    # 이 예제에서는 n=1이므로 상태 큐비트는 1개입니다.
    # 상태 큐비트가 '0'일 때 x[0], '1'일 때 x[1]에 해당한다고 가정합니다.
    estimated_x_relative = np.zeros(2)
    if '0' in state_qubit_probabilities:
         estimated_x_relative[0] = np.sqrt(state_qubit_probabilities['0'])
    if '1' in state_qubit_probabilities:
         estimated_x_relative[1] = np.sqrt(state_qubit_probabilities['1'])

    print(f"추정된 해 벡터 (상대적 크기): {estimated_x_relative}")

    # 추정된 해 벡터의 크기를 고전적인 해의 크기에 맞춰 스케일링합니다.
    # 실제 HHL에서는 이 스케일링 과정이 필요 없지만, 결과를 비교하기 위해 수행합니다.
    # x_classical is assumed to be available from a previous cell's execution
    # If not, you would need to calculate it here: x_classical = np.linalg.solve(A, b)
    try:
        norm_classical_x = np.linalg.norm(x_classical)
    except NameError:
        print("Warning: x_classical not found. Calculating it now.")
        # Assuming A and b are defined in a previous cell
        try:
            x_classical = np.linalg.solve(A, b)
            norm_classical_x = np.linalg.norm(x_classical)
        except NameError:
            print("Error: A or b not found. Cannot calculate classical solution for scaling.")
            norm_classical_x = 1 # Use a default or handle error appropriately


    norm_estimated_x = np.linalg.norm(estimated_x_relative)

    if norm_estimated_x > 0:
        estimated_x = estimated_x_relative * (norm_classical_x / norm_estimated_x)
        print(f"\n추정된 해 벡터 (스케일링됨): {estimated_x}")
    else:
        print("\n추정된 해 벡터를 계산할 수 없습니다 (측정 결과 부족).")

else:
    print("보조 큐비트가 '1'인 측정 결과가 없습니다. 해를 추출할 수 없습니다.")

print("====================================")


------------------------------------
HHL 시뮬레이터 결과 해석
------------------------------------
보조 큐비트가 '1'인 측정 결과: {'10': 2002, '11': 2032}
상태 큐비트 측정 횟수 (보조 큐비트=1): {'0': 2002, '1': 2032}
상태 큐비트 확률 (보조 큐비트=1): {'0': 0.4962816063460585, '1': 0.5037183936539414}
추정된 해 벡터 (상대적 크기): [0.70447257 0.70973121]

추정된 해 벡터 (스케일링됨): [0.70447257 0.70973121]


In [44]:
# 측정 결과를 해석하여 x 벡터를 추출합니다.
# HHL은 첫 번째 큐비트가 '1'로 측정될 확률이 높을수록 해를 얻을 수 있습니다.
# 여기서는 0b100 또는 0b101과 같이 보조 큐비트가 1인 상태의 확률을 봅니다.
# 실제 해는 x = A^-1 * b 입니다.
x_classical = np.linalg.solve(A, b)
print(f"\n고전적인 방법으로 구한 해 (x): {x_classical}")
print("====================================")


고전적인 방법으로 구한 해 (x): [1.5 1.5]
